In [1]:
import InternVideo
import torch
import torch.multiprocessing as mp

# Set the multiprocessing start method to 'spawn'
mp.set_start_method('spawn', force=True)

# Load the pretrained model
model = InternVideo.load_model("../../../InternVideo2/multi_modality/ckpt_new/InternVideo-MM-L-14 (1).ckpt").cuda()


In [2]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import pandas as pd
import os


class SignLanguageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file, header=None, delimiter=' ')
        self.root_dir = root_dir
        self.transform = transform

        # Define the columns since there is no header in the CSV
        self.data.columns = ['video_path', 'label']

        # Create a mapping from string labels to integer indices
        self.label_to_index = {label: idx for idx, label in enumerate(self.data['label'].unique())}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        video_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        label_str = self.data.iloc[idx, 1]

        # Load the video using InternVideo's function or another method, move to GPU
        video = InternVideo.load_video(video_path).cuda()  # Ensure video tensor is on GPU

        # Apply any transformations to the video
        if self.transform:
            video = self.transform(video)

        # Convert the string label to an integer using the mapping
        label = self.label_to_index[label_str]
        label = torch.tensor(label).long().cuda()  # Ensure the label is also on the GPU

        return video, label

# Define transformations without ToTensor
transform = transforms.Compose([
    # Assuming the video is already a tensor, add any tensor-based transforms here
    # For example, normalization (if needed):
    transforms.Normalize(mean=[0.5], std=[0.5])
])


# Create the dataset and dataloader
train_dataset = SignLanguageDataset('annotations/train.csv', 'annotations', transform=transform)
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)



In [ ]:
# Load your dataset
# train_dataset = SignLanguageDataset(csv_file='path/to/your/csv_file.csv', root_dir='path/to/your/video/files')
import torch.optim as optim
import torch.nn as nn


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().cuda()  # Ensure loss is on GPU
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for videos, labels in train_loader:
        # Move inputs and labels to GPU
        videos = videos.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()  # Clear gradients
        outputs = model(videos)  # Forward pass

        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update model parameters

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Optional: Save the trained model's state dict
torch.save(model.state_dict(), 'intern_video_finetuned.pth')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/l/miniconda3/envs/checkpoint/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/l/miniconda3/envs/checkpoint/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'SignLanguageDataset' on <module '__main__' (built-in)>
